### Import Libraries

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

### Configure Session

In [2]:
spark = SparkSession \
    .builder \
    .appName("OpenCommerce") \
    .config("spark.jars", "/ext/lib/postgresql-42.3.4.jar") \
    .getOrCreate()

### Helper Functions

In [3]:
def get_table_from_database(port: int, database: str, table: str):
    return spark.read \
        .format("jdbc") \
        .option("url", f"jdbc:postgresql://host.docker.internal:{port}/{database}?user=postgres") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", table) \
        .load()

### Get Data

In [31]:
activate_locations = get_table_from_database(9037, "activate_core_production", "locations")
oc_addresses = get_table_from_database(9036, "open_commerce", "address").withColumnRenamed("geoLocationLongitude", "lon") \
                                                                        .withColumnRenamed("geoLocationLatitude", "lat")
oc_locations = get_table_from_database(9036, "open_commerce", "location").join(oc_addresses, 
                                                                               col("addressId") == oc_addresses.id,
                                                                               "left") \
                                                                         .drop("createdAt","updatedAt")

In [33]:
display(oc_locations.select("alias").toPandas())

,alias
0,CEFCO #0032
1,CEFCO #1022
2,CEFCO #0069
3,CEFCO #0057
4,CEFCO #1013
...,...
217,CEFCO #1024
218,CEFCO #2012
219,CEFCO #1016
220,CEFCO #0006
